In [ ]:
import pandas as pd

# Load the data from a CSV file
data = pd.read_csv('/content/Combined_Temp_Pressure4.csv')

# Display the first few rows of the data
print(data.head())


In [ ]:
data['Timestamp'] = pd.to_datetime(data['Timestamp'])
data['Time (s)'] = (data['Timestamp'] - data['Timestamp'].min()).dt.total_seconds()

# Extract features and target variable
features = data[['Time (s)', 'Temperature (in Celsius)', 'Pressure (in MPa)']].values

In [ ]:
# Normalize the features
from sklearn.preprocessing import MinMaxScaler
import numpy as np
scaler = MinMaxScaler()
features_normalized = scaler.fit_transform(features)

# Create sequences of the data
def create_sequences(data, seq_length):
    xs = []
    ys = []
    for i in range(len(data) - seq_length):
        x = data[i:i+seq_length]
        y = data[i+seq_length]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

seq_length = 10
X, y = create_sequences(features_normalized, seq_length)

# Split the data into training and testing sets
split_ratio = 0.8
split_index = int(len(X) * split_ratio)
X_train, X_test = X[:split_index], X[split_index:]
y_train, y_test = y[:split_index], y[split_index:]

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
import matplotlib.pyplot as plt


# Build the RNN model
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(seq_length, X.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(50, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(3))

model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
history = model.fit(X_train, y_train, epochs=50, validation_split=0.2)

# Evaluate the model
loss = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}')

# Make predictions
predictions = model.predict(X_test)

# Inverse transform the predictions to get the actual values
predictions_inverse = scaler.inverse_transform(predictions)
y_test_inverse = scaler.inverse_transform(y_test)



In [ ]:
# Extract the time, temperature, and pressure for plotting
time_test = y_test_inverse[:, 0]
actual_temperature = y_test_inverse[:, 1]
predicted_temperature = predictions_inverse[:, 1]
actual_pressure = y_test_inverse[:, 2]
predicted_pressure = predictions_inverse[:, 2]

# Plot actual vs predicted temperature
plt.figure(figsize=(14, 6))

plt.subplot(2, 1, 1)
plt.plot(time_test, actual_temperature, label='Actual Temperature', color='b')
plt.plot(time_test, predicted_temperature, label='Predicted Temperature', color='r')
plt.xlabel('Time (s)')
plt.ylabel('Temperature (in Celsius)')
plt.title('Actual vs Predicted Temperature')
plt.legend()

# Plot actual vs predicted pressure
plt.subplot(2, 1, 2)
plt.plot(time_test, actual_pressure, label='Actual Pressure', color='b')
plt.plot(time_test, predicted_pressure, label='Predicted Pressure', color='r')
plt.xlabel('Time (s)')
plt.ylabel('Pressure (in MPa)')
plt.title('Actual vs Predicted Pressure')
plt.legend()

plt.tight_layout()
plt.show()